## Решите следующие задачи для данных велопарковок Сан-Франциско (trips.csv, stations.csv):

1.	Найти велосипед с максимальным временем пробега.
2.	Найти наибольшее геодезическое расстояние между станциями.
3.	Найти путь велосипеда с максимальным временем пробега через станции.
4.	Найти количество велосипедов в системе.
5.	Найти пользователей потративших на поездки более 3 часов.

In [ ]:
# Найти пользователей потративших на поездки более 3 часов.
users = (trips_df.groupBy("zip_code")
         .agg((F.sum("duration")/3600).alias("sum_duration"))
         .filter(F.col("sum_duration")>3)
         .select("zip_code", F.round("sum_duration",1).alias("sum_duration"))
         .filter(F.col("zip_code")!="nil")
         .orderBy(F.desc("sum_duration"))
).collect()

print(users)

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.appName("lr1").config("spark.sql.legacy.timeParserPolicy", "LEGACY").getOrCreate()

In [3]:
trips_df = spark.read.option("header", "true").option("inferSchema", "true").csv("trip.csv")
print(trips_df.dtypes)

stations_df = spark.read.option("header", "true").option("inferSchema", "true").csv("station.csv")
print(stations_df.dtypes)

In [9]:
# Найти велосипед с максимальным временем пробега.
result =(trips_df.groupBy("bike_id")
         .agg((F.sum("duration")/3600).alias("sum_duration"))
         .orderBy(F.desc("sum_duration"))
         .limit(1)
         .select("bike_id", F.round("sum_duration", 1).alias("duration"))
).collect()
print(result)


In [5]:
# Найти наибольшее геодезическое расстояние между станциями
def calc_dist(lat1, lon1, lat2, lon2):
    R = 6371 # Радиус Земли
    lat1, lat2, lon1, lon2 = F.radians(lat1), F.radians(lat2), F.radians(lon1), F.radians(lon2)
    return R * F.acos(F.sin(lat1) * F.sin(lat2) + F.cos(lat1) * F.cos(lat2) * F.cos(lon2 - lon1))

max_distance = (stations_df.alias("station1")
                .crossJoin(stations_df.alias("station2"))
                .filter(F.col("station1.id") < F.col("station2.id"))
                .withColumn("distance", calc_dist(F.col("station1.lat"), F.col("station1.long"), F.col("station2.lat"), F.col("station2.long")))
                .agg(F.max("distance").alias("max_distance"))
                .collect()[0]["max_distance"])

print(max_distance)

In [6]:
# Найти путь велосипеда с максимальным временем пробега через станции.

# Первое задание
max_duration_bike =(trips_df.groupBy("bike_id")
         .agg((F.sum("duration")/3600).alias("sum_duration"))
         .orderBy(F.desc("sum_duration"))
         .limit(1)
         .select("bike_id", F.round("sum_duration", 1))
).collect()

bike_trips = (trips_df.filter(F.col("bike_id")==max_duration_bike[0]["bike_id"])
             .select("start_date", "start_station_name", "end_date", "end_station_name")
             .orderBy("start_date")
).collect()

print(bike_trips)

In [22]:
# Найти количество велосипедов в системе.
bike_count=(trips_df.groupBy("bike_id").count().count())

print(bike_count)
